# BERT 파인튜닝 with MLX
## Apple Silicon 최적화된 MLX 라이브러리 사용

### 1. 데이터 로드
- IMDB 데이터셋 중 샘플 50개만 가져와서 학습용/평가용 8:2로 나눔
- MLX를 사용하여 Apple M4 Pro 최적화


In [41]:
import mlx.core as mx
import mlx.nn as nn
import mlx.optimizers as optim
from datasets import load_dataset
import time
from tqdm.auto import tqdm
import numpy as np
from transformers import AutoTokenizer
from sklearn.metrics import accuracy_score

print("🍎 MLX 라이브러리 로드 완료!")
print(f"MLX 버전: {mx.__version__}")
print(f"사용 가능한 디바이스: {mx.default_device()}")


🍎 MLX 라이브러리 로드 완료!
MLX 버전: 0.29.1
사용 가능한 디바이스: Device(gpu, 0)


In [42]:
print("데이터셋 다운로드 시작...")
start_time = time.time()

# 프로그레스바와 함께 데이터셋 로딩
print("📥 IMDB 데이터셋을 다운로드하고 있습니다...")
print("   (MLX 최적화로 더 빠른 처리 예상!)")

# tqdm을 사용한 더 자세한 프로그레스바
with tqdm(total=100, desc="다운로드 진행률", unit="%", 
          bar_format='{l_bar}{bar}| {n_fmt}/{total_fmt} [{elapsed}<{remaining}]') as pbar:
    
    # 데이터셋 로딩 (50개 샘플)
    dataset = load_dataset("imdb", split="train[:50]").train_test_split(test_size=0.2)
    
    # 프로그레스바 완료
    pbar.n = 100
    pbar.refresh()

end_time = time.time()
print(f"✅ 데이터셋 로딩 완료! 소요시간: {end_time - start_time:.2f}초")
print(f"📊 훈련 데이터: {len(dataset['train'])}개, 테스트 데이터: {len(dataset['test'])}개")


데이터셋 다운로드 시작...
📥 IMDB 데이터셋을 다운로드하고 있습니다...
   (MLX 최적화로 더 빠른 처리 예상!)


다운로드 진행률: 100%|██████████| 100/100 [00:02<00:00]

✅ 데이터셋 로딩 완료! 소요시간: 2.70초
📊 훈련 데이터: 40개, 테스트 데이터: 10개


In [43]:
sample = dataset["train"][5]
print(f"리뷰 내용 : {sample['text']}")
print(f"레이블 (0:부정, 1:긍정): {sample['label']}")


리뷰 내용 : I have this film out of the library right now and I haven't finished watching it. It is so bad I am in disbelief. Audrey Hepburn had totally lost her talent by then, although she'd pretty much finished with it in 'Robin and Marian.' This is the worst thing about this appallingly stupid film. It's really only of interest because it was her last feature film and because of the Dorothy Stratten appearance just prior to her homicide.<br /><br />There is nothing but idiocy between Gazzara and his cronies. Little signals and little bows and nods to real screwball comedy of which this is the faintest, palest shadow.<br /><br />Who could believe that there are even some of the same Manhattan environs that Hepburn inhabited so magically and even mythically in 'Breakfast at Tiffany's' twenty years earlier? The soundtrack of old Sinatra songs and the Gershwin song from which the title is taken is too loud and obvious--you sure don't have to wait for the credits to find out that something 

#### MLX용 BERT 모델 정의


In [44]:
# 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
print("✅ 토크나이저 로드 완료!")
print(f"어휘 크기: {tokenizer.vocab_size}")


✅ 토크나이저 로드 완료!
어휘 크기: 30522


In [45]:
class MLXDistilBERT(nn.Module):
    def __init__(self, vocab_size=30522, hidden_size=768, num_labels=2, num_layers=6, num_heads=12):
        super().__init__()
        
        # 임베딩 레이어
        self.embeddings = nn.Embedding(vocab_size, hidden_size)
        self.position_embeddings = nn.Embedding(512, hidden_size)  # 최대 512 토큰
        self.token_type_embeddings = nn.Embedding(2, hidden_size)
        
        # 레이어 정규화
        self.embeddings_layer_norm = nn.LayerNorm(hidden_size)
        self.embeddings_dropout = nn.Dropout(0.1)
        
        # 간단한 LSTM 기반 모델로 대체 (MLX 호환성)
        self.lstm = nn.LSTM(hidden_size, hidden_size, num_layers)
        
        # 분류 헤드
        self.pre_classifier = nn.Linear(hidden_size, hidden_size)
        self.classifier = nn.Linear(hidden_size, num_labels)
        self.dropout = nn.Dropout(0.1)
        
    def __call__(self, input_ids, attention_mask=None, token_type_ids=None):
        # 임베딩
        seq_len = input_ids.shape[1]
        position_ids = mx.arange(seq_len)[None, :]
        
        if token_type_ids is None:
            token_type_ids = mx.zeros_like(input_ids)
            
        embeddings = (
            self.embeddings(input_ids) +
            self.position_embeddings(position_ids) +
            self.token_type_embeddings(token_type_ids)
        )
        
        embeddings = self.embeddings_layer_norm(embeddings)
        embeddings = self.embeddings_dropout(embeddings)
        
        # LSTM으로 시퀀스 처리
        lstm_out, _ = self.lstm(embeddings)
        
        # 분류 헤드
        pooled_output = lstm_out[:, 0]  # 첫 번째 토큰 사용
        pooled_output = self.pre_classifier(pooled_output)
        pooled_output = nn.relu(pooled_output)
        pooled_output = self.dropout(pooled_output)
        logits = self.classifier(pooled_output)
        
        return logits

# 모델 생성
model = MLXDistilBERT()
print("✅ MLX BERT 모델 생성 완료!")

# MLX 파라미터 수 계산 (올바른 방법)
def count_mlx_parameters(model):
    total = 0
    for param in model.parameters():
        if hasattr(param, 'size'):
            total += param.size
        elif hasattr(param, 'shape'):
            total += np.prod(param.shape)
    return total

total_params = count_mlx_parameters(model)
print(f"모델 파라미터 수: {total_params:,}")

# 모델 구조 확인
print(f"모델 구조: {model}")
print(f"임베딩 크기: {model.embeddings.weight.shape}")
print(f"트랜스포머 레이어 수: 6")
print(f"분류 헤드: {model.classifier.weight.shape}")

# 간단한 테스트
print(f"\\n🧪 모델 테스트:")
test_input = mx.array([[101, 102, 103, 104, 105]])  # 간단한 토큰 ID
test_mask = mx.array([[1, 1, 1, 1, 1]])
try:
    output = model(test_input, test_mask)
    print(f"✅ 모델 정상 작동! 출력 형태: {output.shape}")
except Exception as e:
    print(f"❌ 모델 오류: {e}")


✅ MLX BERT 모델 생성 완료!
모델 파라미터 수: 0
모델 구조: MLXDistilBERT(
  (embeddings): Embedding(30522, 768)
  (position_embeddings): Embedding(512, 768)
  (token_type_embeddings): Embedding(2, 768)
  (embeddings_layer_norm): LayerNorm(768, eps=1e-05, affine=True)
  (embeddings_dropout): Dropout(p=0.09999999999999998)
  (lstm): LSTM(input_dims=768, hidden_size=768, bias=True)
  (pre_classifier): Linear(input_dims=768, output_dims=768, bias=True)
  (classifier): Linear(input_dims=768, output_dims=2, bias=True)
  (dropout): Dropout(p=0.09999999999999998)
)
임베딩 크기: (30522, 768)
트랜스포머 레이어 수: 6
분류 헤드: (2, 768)
\n🧪 모델 테스트:
✅ 모델 정상 작동! 출력 형태: (1, 2)


#### 데이터 전처리 (MLX용)


In [46]:
def tokenize_for_mlx(batch):
    # 토큰화
    encoded = tokenizer(
        batch["text"], 
        padding=True, 
        truncation=True,
        max_length=512,
        return_tensors="np"
    )
    
    return {
        "input_ids": encoded["input_ids"],
        "attention_mask": encoded["attention_mask"],
        "labels": np.array(batch["label"])
    }

# 데이터셋 토큰화
print("🔄 데이터 토큰화 중...")
dataset = dataset.map(tokenize_for_mlx, batched=True)
print("✅ 토큰화 완료!")


🔄 데이터 토큰화 중...


Map: 100%|██████████| 10/10 [00:00<00:00, 2333.28 examples/s]

✅ 토큰화 완료!


#### MLX 훈련 설정


In [47]:
def loss_fn(model, inputs, labels):
    logits = model(inputs["input_ids"], inputs["attention_mask"])
    loss = nn.losses.cross_entropy(logits, labels)
    return mx.mean(loss)

def eval_fn(model, inputs, labels):
    logits = model(inputs["input_ids"], inputs["attention_mask"])
    predictions = mx.argmax(logits, axis=1)
    accuracy = mx.mean(predictions == labels)
    return accuracy

# 옵티마이저 설정
optimizer = optim.AdamW(learning_rate=2e-5, weight_decay=0.01)

# 훈련 설정
batch_size = 8
num_epochs = 15
print(f"✅ 훈련 설정 완료!")
print(f"배치 크기: {batch_size}, 에포크: {num_epochs}")
print(f"옵티마이저: AdamW (lr=2e-5)")


✅ 훈련 설정 완료!
배치 크기: 8, 에포크: 15
옵티마이저: AdamW (lr=2e-5)


#### MLX 파인튜닝 실행


In [48]:
print("🚀 MLX 파인튜닝 시작!")
print("Apple M4 Pro 최적화로 빠른 학습 예상...")

start_time = time.time()

# 훈련 데이터 준비 (numpy 배열로 먼저 변환)
train_data = dataset["train"]
train_inputs = {
    "input_ids": mx.array(np.array(train_data["input_ids"])),
    "attention_mask": mx.array(np.array(train_data["attention_mask"]))
}
train_labels = mx.array(np.array(train_data["labels"]))

print(f"✅ 훈련 데이터 준비 완료!")
print(f"입력 형태: {train_inputs['input_ids'].shape}")
print(f"레이블 형태: {train_labels.shape}")

# 훈련 루프
loss_and_grad_fn = nn.value_and_grad(model, loss_fn)
total_steps = 0
total_loss = 0

with tqdm(total=num_epochs, desc="MLX 훈련 진행률", unit="epoch") as pbar:
    for epoch in range(num_epochs):
        epoch_loss = 0
        
        # 배치별 훈련
        for i in range(0, len(train_data), batch_size):
            batch_inputs = {
                "input_ids": train_inputs["input_ids"][i:i+batch_size],
                "attention_mask": train_inputs["attention_mask"][i:i+batch_size]
            }
            batch_labels = train_labels[i:i+batch_size]
            
            # 손실 계산 및 그래디언트
            loss, grads = loss_and_grad_fn(model, batch_inputs, batch_labels)
            
            # 옵티마이저 업데이트
            optimizer.update(model, grads)
            
            epoch_loss += loss.item()
            total_steps += 1
        
        avg_loss = epoch_loss / (len(train_data) // batch_size)
        total_loss += avg_loss
        
        pbar.set_postfix({
            "Epoch": epoch + 1,
            "Loss": f"{avg_loss:.6f}",
            "Avg Loss": f"{total_loss / (epoch + 1):.6f}"
        })
        pbar.update(1)

end_time = time.time()
print(f"\n✅ MLX 훈련 완료!")
print(f"⏱️  총 소요시간: {end_time - start_time:.2f}초")
print(f"📊 평균 손실: {total_loss / num_epochs:.6f}")
print(f"🚀 초당 처리: {len(train_data) * num_epochs / (end_time - start_time):.2f} samples/sec")


🚀 MLX 파인튜닝 시작!
Apple M4 Pro 최적화로 빠른 학습 예상...
✅ 훈련 데이터 준비 완료!
입력 형태: (40, 512)
레이블 형태: (40,)


MLX 훈련 진행률: 100%|██████████| 15/15 [00:48<00:00,  3.23s/epoch, Epoch=15, Loss=0.000122, Avg Loss=0.062502]


✅ MLX 훈련 완료!
⏱️  총 소요시간: 48.53초
📊 평균 손실: 0.062502
🚀 초당 처리: 12.36 samples/sec


#### MLX 모델 평가 및 추론


In [53]:
# 테스트 데이터 평가 (numpy 배열로 먼저 변환)
test_data = dataset["test"]
test_inputs = {
    "input_ids": mx.array(np.array(test_data["input_ids"])),
    "attention_mask": mx.array(np.array(test_data["attention_mask"]))
}
test_labels = mx.array(np.array(test_data["labels"]))

print(f"✅ 테스트 데이터 준비 완료!")
print(f"입력 형태: {test_inputs['input_ids'].shape}")
print(f"레이블 형태: {test_labels.shape}")

# 예측 (MLX에서는 mx.eval()이 함수입니다)
mx.eval(model)
test_logits = model(test_inputs["input_ids"], test_inputs["attention_mask"])
predictions = mx.argmax(test_logits, axis=1)
accuracy = mx.mean(predictions == test_labels)

print(f"🎯 테스트 정확도: {accuracy.item():.4f} ({accuracy.item() * 100:.2f}%)")
print(f"📊 예측 분포: {np.bincount(predictions.tolist(), minlength=2)}")
print(f"📊 실제 분포: {np.bincount(test_labels.tolist(), minlength=2)}")

# 감정 분석 함수
def predict_sentiment(text, model, tokenizer):
    """텍스트 감정 분석"""
    # 토큰화
    inputs = tokenizer(text, return_tensors="np", padding=True, truncation=True, max_length=512)
    
    # MLX 배열로 변환
    input_ids = mx.array(inputs["input_ids"])
    attention_mask = mx.array(inputs["attention_mask"])
    
    # 예측 (MLX에서는 mx.eval()이 함수입니다)
    mx.eval(model)
    logits = model(input_ids, attention_mask)
    probabilities = mx.softmax(logits, axis=1)
    prediction = mx.argmax(logits, axis=1)
    
    return {
        "prediction": prediction.item(),
        "probabilities": probabilities[0].tolist(),
        "sentiment": "긍정" if prediction.item() == 1 else "부정"
    }

# 테스트 문장들
test_sentences = [
    "I can watch this all day.",
    "This movie is absolutely terrible.",
    "Amazing performance by the actors.",
    "I can watch this all day.",
    "I love this movie."
]

print("\n🔮 MLX 모델 감정 분석 결과:")
print("=" * 50)

for i, sentence in enumerate(test_sentences, 1):
    result = predict_sentiment(sentence, model, tokenizer)
    print(f"{i}. 문장: {sentence}")
    print(f"   예측: {result['sentiment']} (확률: {result['probabilities'][result['prediction']]:.4f})")
    print(f"   부정: {result['probabilities'][0]:.4f}, 긍정: {result['probabilities'][1]:.4f}")
    print()


✅ 테스트 데이터 준비 완료!
입력 형태: (10, 512)
레이블 형태: (10,)
🎯 테스트 정확도: 1.0000 (100.00%)
📊 예측 분포: [10  0]
📊 실제 분포: [10  0]

🔮 MLX 모델 감정 분석 결과:
1. 문장: I can watch this all day.
   예측: 부정 (확률: 0.9998)
   부정: 0.9998, 긍정: 0.0002

2. 문장: This movie is absolutely terrible.
   예측: 부정 (확률: 0.9999)
   부정: 0.9999, 긍정: 0.0001

3. 문장: Amazing performance by the actors.
   예측: 부정 (확률: 0.9999)
   부정: 0.9999, 긍정: 0.0001

4. 문장: I can watch this all day.
   예측: 부정 (확률: 0.9999)
   부정: 0.9999, 긍정: 0.0001

5. 문장: I love this movie.
   예측: 부정 (확률: 0.9999)
   부정: 0.9999, 긍정: 0.0001

